In [ ]:
!pip install -U ultralytics torch torchvision optuna

In [ ]:
from ultralytics import YOLO
import torch
import os
import optuna
import json

In [ ]:
# Configuración de rutas
ROOT = "Dataset_Final"

DATA_YAML = os.path.join(ROOT, "data.yaml")
YOLO_ROOT = os.path.join(ROOT, "Yolo_Experiments")
os.makedirs(YOLO_ROOT, exist_ok=True)

In [ ]:
DEVICE = 0 if torch.cuda.is_available() else 'cpu'
print("Device:", DEVICE)


In [ ]:
# Número de clases
NUM_CLASSES = 4  # door, window, balcony, railing

In [ ]:
def objective(trial):
    # Hyperparametros ajustables 
    yolo_version = trial.suggest_categorical("yolo_version", ["yolov8s.pt", "yolov8m.pt", "yolov8l.pt"])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW"])

    trial_dir = os.path.join(YOLO_ROOT, f"trial_{trial.number}")
    os.makedirs(trial_dir, exist_ok=True)

    # Modelo
    model = YOLO(yolo_version)
    model.freeze(backbone=True)  # backbone congelado, solo entrenar la cabeza 

    epochs = 10  
    for epoch in range(1, epochs + 1):
        model.train(
            data=DATA_YAML,
            epochs=1,    # 1 epoch por loop para evaluar métricas intermedias y permitir pruning
            imgsz=640,
            batch=8,
            lr0=lr,
            optimizer=optimizer_name,
            device=DEVICE,
            project=YOLO_ROOT,
            name=f"trial_{trial.number}",
            save=False,
            cache=True
        )

        # Evaluación intermedia y pruning 
        if epoch % 2 == 0:  # evaluar cada 2 epochs
            results = model.val()
            map50 = results['metrics/mAP_0.5']

            if epoch >= 4:  # pruning activo desde epoch 4
                trial.report(1 - map50, epoch)
                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

    # Guardar pesos finales
    model.save(os.path.join(trial_dir, "best.pt"))

    # Evaluación final
    results = model.val()
    map50 = results['metrics/mAP_0.5']
    with open(os.path.join(trial_dir, "metrics.json"), "w") as f:
        json.dump(results, f, indent=4)

    return 1 - map50


In [ ]:
# Crear estudio con Successive Halving Pruner
pruner = optuna.pruners.SuccessiveHalvingPruner()
study = optuna.create_study(direction="minimize", pruner=pruner)

# Ejecutar Optuna
study.optimize(objective, n_trials=20)  

print("Mejor configuración encontrada:")
print(study.best_params)


In [ ]:
# Cargar el mejor trial
best_trial_number = study.best_trial.number
best_trial_dir = os.path.join(YOLO_ROOT, f"trial_{best_trial_number}")

# Cargar modelo YOLO
from ultralytics import YOLO
best_model = YOLO(os.path.join(best_trial_dir, "best.pt"))

# Evaluar validación final
results = best_model.val()
print(results['metrics/mAP_0.5'])
